In [1]:
# from pycaret.regression import setup, compare_models, evaluate_model, predict_model, save_model, load_model

ModuleNotFoundError: No module named 'pycaret'

In [30]:
import pandas as pd
import numpy as np
from math import sqrt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [31]:
weather = pd.read_csv('../Data/data_fix_temp.txt')

In [32]:
# left_skewed_cols = ["dew2m.degC.", "rh2m...", "lcloud...", "surpre.Pa.", "wamix.kg.kg.", "outlr.W.m2.", "rh950...", "rh800...", "rh500..."]
# for col in left_skewed_cols:
#     weather[col], _ = stats.boxcox(weather[col] + 1)  

# right_skewed_cols = ["wspeed.m.s.", "pblh.m.", "lifcl.m.",  "ws950.m.s.", "ws800.m.s.", "ws500.m.s."]
# for col in right_skewed_cols:
#     weather[col] = np.log(weather[col])  

# bimodal_cols = ["wd500.deg.", "wd800.deg.", "wd950.deg."]
# num_bins = 5
# for col in bimodal_cols:
#     weather[col] = pd.cut(weather[col], bins=num_bins, labels=False)

In [33]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 544274 entries, 1 to 544274
Data columns (total 39 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          544274 non-null  object 
 1   lokasi        544274 non-null  int64  
 2   con_prec.mm.  544274 non-null  float64
 3   mic_prec.mm.  544274 non-null  float64
 4   sha_prec.mm.  544274 non-null  int64  
 5   suhu2m.degC.  544274 non-null  float64
 6   dew2m.degC.   544274 non-null  float64
 7   rh2m...       544274 non-null  float64
 8   wspeed.m.s.   544274 non-null  float64
 9   wdir.deg.     544274 non-null  float64
 10  lcloud...     544274 non-null  float64
 11  mcloud...     544274 non-null  float64
 12  hcloud...     544274 non-null  float64
 13  surpre.Pa.    544274 non-null  float64
 14  clmix.kg.kg.  544274 non-null  float64
 15  wamix.kg.kg.  544274 non-null  float64
 16  outlr.W.m2.   544274 non-null  float64
 17  pblh.m.       544274 non-null  float64
 18  lifcl.m. 

In [34]:
weather.describe()

,lokasi,con_prec.mm.,mic_prec.mm.,sha_prec.mm.,suhu2m.degC.,dew2m.degC.,rh2m...,wspeed.m.s.,wdir.deg.,lcloud...,...,t500.degC.,rh500...,ws500.m.s.,wd500.deg.,hari,UTC,LAT,LON,ELEV,t_obs
count,544274.000000,544274.000000,5.442740e+05,544274.0,544274.000000,544274.000000,544274.000000,544274.000000,544274.000000,544274.000000,...,544274.000000,544274.000000,544274.000000,544274.000000,544274.000000,544274.0,544274.000000,544274.000000,544274.000000,544274.000000
mean,96884.176938,0.625100,1.633177e-01,0.0,26.238859,22.720439,82.063592,2.591375,181.631337,0.607279,...,-5.047452,67.607710,5.349764,144.651146,0.160974,12.0,-3.041791,115.316266,102.723196,27.188873
std,588.248982,1.715435,1.804061e+00,0.0,2.880975,2.107988,11.518075,1.716567,98.590547,0.273119,...,0.750793,22.168508,2.749737,87.580012,0.367507,0.0,4.024515,11.472831,269.274703,3.117043
min,96001.000000,0.000000,0.000000e+00,0.0,11.394600,3.866710,23.164500,0.003070,0.000916,0.000000,...,-8.813010,0.000015,0.089455,0.145816,0.000000,12.0,-10.766620,95.337850,0.000000,9.700000
25%,96297.000000,0.000000,0.000000e+00,0.0,24.272600,21.946800,74.607950,1.293880,103.649250,0.433848,...,-5.539040,56.068250,3.244365,81.348200,0.000000,12.0,-6.700000,106.422000,5.000000,25.000000
50%,96855.000000,0.013227,0.000000e+00,0.0,26.546600,23.210800,83.399600,2.235905,176.662000,0.645353,...,-5.056590,72.620050,5.000060,107.554000,0.000000,12.0,-3.045240,114.355300,15.000000,26.900000
75%,97282.000000,0.318031,4.262848e-09,0.0,28.278975,24.120200,90.933500,3.530938,266.124750,0.818182,...,-4.568865,84.478875,7.078793,216.558000,0.000000,12.0,-0.142060,122.852500,50.000000,29.600000
max,99992.000000,35.757000,1.141500e+02,0.0,36.192300,27.283400,100.000000,22.739000,359.998000,1.000000,...,-0.671190,100.000000,25.247400,359.682000,1.000000,12.0,5.876550,140.720000,1653.000000,37.200000


In [35]:
weather.head()

,Date,lokasi,con_prec.mm.,mic_prec.mm.,sha_prec.mm.,suhu2m.degC.,dew2m.degC.,rh2m...,wspeed.m.s.,wdir.deg.,...,t500.degC.,rh500...,ws500.m.s.,wd500.deg.,hari,UTC,LAT,LON,ELEV,t_obs
1,2021-02-27 15:00:00,96001,0.0,0.0,0,27.3973,21.6059,70.8014,3.43326,12.4580,...,-5.20890,11.0889,6.22327,57.5841,0,12,5.87655,95.33785,126,23.2
2,2021-02-27 18:00:00,96001,0.0,0.0,0,27.0090,21.5161,72.0376,3.04901,27.2450,...,-5.05963,12.7044,5.96761,59.9813,0,12,5.87655,95.33785,126,23.0
3,2021-02-27 21:00:00,96001,0.0,0.0,0,26.5672,21.4226,73.5149,2.04556,37.3569,...,-4.89987,15.2381,7.04134,67.3222,0,12,5.87655,95.33785,126,21.6
4,2021-02-28 00:00:00,96001,0.0,0.0,0,26.4519,21.5400,74.5507,2.22508,20.1816,...,-4.72228,16.3007,7.54606,65.4942,0,12,5.87655,95.33785,126,19.8
5,2021-02-28 03:00:00,96001,0.0,0.0,0,26.7230,21.3691,72.6038,2.34268,14.7934,...,-4.64246,19.0442,8.93947,73.4932,0,12,5.87655,95.33785,126,28.4


In [36]:
y = weather.copy()
y = weather[['Date', 't_obs']]
y=y.set_index('Date')
y

,t_obs
Date,
2021-02-27 15:00:00,23.2
2021-02-27 18:00:00,23.0
2021-02-27 21:00:00,21.6
2021-02-28 00:00:00,19.8
2021-02-28 03:00:00,28.4
...,...
2023-04-12 03:00:00,31.8
2023-04-12 06:00:00,35.0
2023-04-12 09:00:00,30.6


In [37]:
weather_filtered = weather.drop(columns=['sha_prec.mm.', 'clmix.kg.kg.', 'mdbz', 'hari', 'UTC', 'LON', 'LAT', 't_obs'])
weather_filtered

,Date,lokasi,con_prec.mm.,mic_prec.mm.,suhu2m.degC.,dew2m.degC.,rh2m...,wspeed.m.s.,wdir.deg.,lcloud...,...,wd950.deg.,t800.degC.,rh800...,ws800.m.s.,wd800.deg.,t500.degC.,rh500...,ws500.m.s.,wd500.deg.,ELEV
1,2021-02-27 15:00:00,96001,0.000000,0.000000e+00,27.3973,21.6059,70.8014,3.433260,12.4580,0.553781,...,12.6254,17.0906,20.4601,0.767220,281.622,-5.20890,11.0889,6.22327,57.5841,126
2,2021-02-27 18:00:00,96001,0.000000,0.000000e+00,27.0090,21.5161,72.0376,3.049010,27.2450,0.640686,...,26.4370,17.4508,16.8533,0.653591,250.969,-5.05963,12.7044,5.96761,59.9813,126
3,2021-02-27 21:00:00,96001,0.000000,0.000000e+00,26.5672,21.4226,73.5149,2.045560,37.3569,0.717464,...,34.4898,17.3211,16.4506,0.823568,227.030,-4.89987,15.2381,7.04134,67.3222,126
4,2021-02-28 00:00:00,96001,0.000000,0.000000e+00,26.4519,21.5400,74.5507,2.225080,20.1816,0.698006,...,23.7710,16.8798,18.1771,0.492229,120.120,-4.72228,16.3007,7.54606,65.4942,126
5,2021-02-28 03:00:00,96001,0.000000,0.000000e+00,26.7230,21.3691,72.6038,2.342680,14.7934,0.626203,...,23.7966,16.8302,18.2255,1.688550,137.419,-4.64246,19.0442,8.93947,73.4932,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544270,2023-04-12 03:00:00,99992,0.003601,0.000000e+00,29.7642,23.4001,68.8035,3.721340,261.5040,0.390150,...,257.7270,15.4080,86.8257,6.484750,260.471,-4.69787,73.0758,2.29511,231.9670,41
544271,2023-04-12 06:00:00,99992,3.669660,7.573590e-21,30.7901,24.1140,67.7143,5.811930,263.9440,0.536106,...,256.4770,15.5844,86.8180,7.014090,248.417,-4.63960,75.7922,2.05083,268.5790,41
544272,2023-04-12 09:00:00,99992,4.045040,2.640270e-09,28.8450,24.1964,76.1178,5.839920,268.8950,0.229447,...,257.0560,15.7292,83.2283,8.068280,248.233,-4.40575,77.1489,2.52709,272.9220,41
544273,2023-04-12 12:00:00,99992,0.000000,0.000000e+00,25.8334,23.9709,89.6050,0.500122,210.7490,0.087511,...,239.4350,15.8302,80.2176,7.675180,256.296,-4.33471,76.3170,3.07955,259.2500,41


In [38]:
weather_filtered_sorted = weather_filtered.set_index('Date')
weather_filtered_sorted

,lokasi,con_prec.mm.,mic_prec.mm.,suhu2m.degC.,dew2m.degC.,rh2m...,wspeed.m.s.,wdir.deg.,lcloud...,mcloud...,...,wd950.deg.,t800.degC.,rh800...,ws800.m.s.,wd800.deg.,t500.degC.,rh500...,ws500.m.s.,wd500.deg.,ELEV
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-27 15:00:00,96001,0.000000,0.000000e+00,27.3973,21.6059,70.8014,3.433260,12.4580,0.553781,0.000000,...,12.6254,17.0906,20.4601,0.767220,281.622,-5.20890,11.0889,6.22327,57.5841,126
2021-02-27 18:00:00,96001,0.000000,0.000000e+00,27.0090,21.5161,72.0376,3.049010,27.2450,0.640686,0.000000,...,26.4370,17.4508,16.8533,0.653591,250.969,-5.05963,12.7044,5.96761,59.9813,126
2021-02-27 21:00:00,96001,0.000000,0.000000e+00,26.5672,21.4226,73.5149,2.045560,37.3569,0.717464,0.000000,...,34.4898,17.3211,16.4506,0.823568,227.030,-4.89987,15.2381,7.04134,67.3222,126
2021-02-28 00:00:00,96001,0.000000,0.000000e+00,26.4519,21.5400,74.5507,2.225080,20.1816,0.698006,0.000000,...,23.7710,16.8798,18.1771,0.492229,120.120,-4.72228,16.3007,7.54606,65.4942,126
2021-02-28 03:00:00,96001,0.000000,0.000000e+00,26.7230,21.3691,72.6038,2.342680,14.7934,0.626203,0.000000,...,23.7966,16.8302,18.2255,1.688550,137.419,-4.64246,19.0442,8.93947,73.4932,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-12 03:00:00,99992,0.003601,0.000000e+00,29.7642,23.4001,68.8035,3.721340,261.5040,0.390150,0.488739,...,257.7270,15.4080,86.8257,6.484750,260.471,-4.69787,73.0758,2.29511,231.9670,41
2023-04-12 06:00:00,99992,3.669660,7.573590e-21,30.7901,24.1140,67.7143,5.811930,263.9440,0.536106,0.653845,...,256.4770,15.5844,86.8180,7.014090,248.417,-4.63960,75.7922,2.05083,268.5790,41
2023-04-12 09:00:00,99992,4.045040,2.640270e-09,28.8450,24.1964,76.1178,5.839920,268.8950,0.229447,0.570225,...,257.0560,15.7292,83.2283,8.068280,248.233,-4.40575,77.1489,2.52709,272.9220,41


In [39]:
weather_filtered_sorted.dtypes

lokasi            int64
con_prec.mm.    float64
mic_prec.mm.    float64
suhu2m.degC.    float64
dew2m.degC.     float64
rh2m...         float64
wspeed.m.s.     float64
wdir.deg.       float64
lcloud...       float64
mcloud...       float64
hcloud...       float64
surpre.Pa.      float64
wamix.kg.kg.    float64
outlr.W.m2.     float64
pblh.m.         float64
lifcl.m.        float64
cape.j.kg.      float64
t950.degC.      float64
rh950...        float64
ws950.m.s.      float64
wd950.deg.      float64
t800.degC.      float64
rh800...        float64
ws800.m.s.      float64
wd800.deg.      float64
t500.degC.      float64
rh500...        float64
ws500.m.s.      float64
wd500.deg.      float64
ELEV              int64
dtype: object

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(weather_filtered_sorted, y, test_size=0.1, random_state=42, shuffle=False)

# Now, X_train and y_train are your training data, and X_test and y_test are your testing data.

In [41]:
X_train

,lokasi,con_prec.mm.,mic_prec.mm.,suhu2m.degC.,dew2m.degC.,rh2m...,wspeed.m.s.,wdir.deg.,lcloud...,mcloud...,...,wd950.deg.,t800.degC.,rh800...,ws800.m.s.,wd800.deg.,t500.degC.,rh500...,ws500.m.s.,wd500.deg.,ELEV
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-27 15:00:00,96001,0.000000,0.000000,27.3973,21.6059,70.8014,3.433260,12.4580,0.553781,0.000000,...,12.6254,17.0906,20.4601,0.767220,281.622,-5.20890,11.0889,6.22327,57.5841,126
2021-02-27 18:00:00,96001,0.000000,0.000000,27.0090,21.5161,72.0376,3.049010,27.2450,0.640686,0.000000,...,26.4370,17.4508,16.8533,0.653591,250.969,-5.05963,12.7044,5.96761,59.9813,126
2021-02-27 21:00:00,96001,0.000000,0.000000,26.5672,21.4226,73.5149,2.045560,37.3569,0.717464,0.000000,...,34.4898,17.3211,16.4506,0.823568,227.030,-4.89987,15.2381,7.04134,67.3222,126
2021-02-28 00:00:00,96001,0.000000,0.000000,26.4519,21.5400,74.5507,2.225080,20.1816,0.698006,0.000000,...,23.7710,16.8798,18.1771,0.492229,120.120,-4.72228,16.3007,7.54606,65.4942,126
2021-02-28 03:00:00,96001,0.000000,0.000000,26.7230,21.3691,72.6038,2.342680,14.7934,0.626203,0.000000,...,23.7966,16.8302,18.2255,1.688550,137.419,-4.64246,19.0442,8.93947,73.4932,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-25 09:00:00,97694,1.824930,0.000573,24.8420,23.8414,94.3164,0.642705,15.6097,0.676878,1.000000,...,357.1910,15.3564,97.3721,2.936650,181.543,-3.46006,70.8187,4.94492,253.9400,20
2023-03-25 12:00:00,97694,0.499534,0.075999,24.5510,21.4962,83.2483,0.467697,294.4230,0.476988,0.885464,...,211.1180,15.3637,93.3894,2.731190,209.029,-4.25377,83.6754,3.14305,227.6060,20
2023-03-25 21:00:00,97694,0.000000,0.000000,22.9920,20.2019,84.4479,0.360673,100.4300,0.156663,0.462683,...,196.5610,16.7897,74.4018,1.215510,268.382,-4.97732,82.3028,4.05421,223.1900,20


In [42]:
y_train

,t_obs
Date,
2021-02-27 15:00:00,23.2
2021-02-27 18:00:00,23.0
2021-02-27 21:00:00,21.6
2021-02-28 00:00:00,19.8
2021-02-28 03:00:00,28.4
...,...
2023-03-25 09:00:00,27.6
2023-03-25 12:00:00,25.8
2023-03-25 21:00:00,24.6


In [43]:
scaler = MinMaxScaler()

In [44]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [45]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor

In [48]:
models = {
    'GB Regressor': GradientBoostingRegressor(),
    'ExtraTreesRegressor': ExtraTreesRegressor(),
    'RF Regressor': RandomForestRegressor(),
    'CatBoost': CatBoostRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor()
}
models

{'GB Regressor': GradientBoostingRegressor(),
 'ExtraTreesRegressor': ExtraTreesRegressor(),
 'RF Regressor': RandomForestRegressor(),
 'CatBoost': <catboost.core.CatBoostRegressor at 0x276594396c0>,
 'XGBoost': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
 'LightGBM': LGBMRegressor()}

In [50]:
model_names = []
model_mse_mean = []
for model_name, model in models.items():
    print(f'Starting to crossval {model_name}')
    mses = cross_val_score(model, X_train_scaled, y_train, scoring='neg_mean_squared_error', n_jobs=5, verbose=3)
    model_names.append(model_name)
    model_mse_mean.append(-mses.mean())
    print(f'Finished crossval {model_name} with Neg MSE of {mses.mean}')

Starting to crossval GB Regressor


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 11.4min remaining: 17.0min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 11.4min finished


Finished crossval GB Regressor with Neg MSE of <built-in method mean of numpy.ndarray object at 0x000002764D6EAE50>
Starting to crossval ExtraTreesRegressor


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 11.3min remaining: 16.9min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 11.3min finished


Finished crossval ExtraTreesRegressor with Neg MSE of <built-in method mean of numpy.ndarray object at 0x000002764D6EB330>
Starting to crossval RF Regressor


...

Finished crossval CatBoost with Neg MSE of <built-in method mean of numpy.ndarray object at 0x000002764D6EB870>
Starting to crossval XGBoost


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    6.7s remaining:   10.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    6.8s finished


Finished crossval XGBoost with Neg MSE of <built-in method mean of numpy.ndarray object at 0x000002764D6EAEB0>
Starting to crossval LightGBM


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    4.9s remaining:    7.4s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    5.1s finished


Finished crossval LightGBM with Neg MSE of <built-in method mean of numpy.ndarray object at 0x000002764D6EB690>


In [51]:
model_comparison = pd.DataFrame()
model_comparison['model'] = model_names
model_comparison['mean mse'] = model_mse_mean
model_comparison.sort_values(by=['mean mse'])

,model,mean mse
3,CatBoost,2.605885
1,ExtraTreesRegressor,2.632405
5,LightGBM,2.633128
0,GB Regressor,2.709862
2,RF Regressor,2.713086
4,XGBoost,2.719832


In [66]:
optimizer_results = pd.DataFrame(grid_search.cv_results_)
optimizer_results[['param_depth', 'param_learning_rate', 'param_iterations', 'mean_test_score']]

,param_depth,param_learning_rate,param_iterations,mean_test_score
0,6,0.01,50,-4.633620
1,6,0.05,50,-3.754629
2,6,0.1,50,-3.540262
3,6,0.01,100,-4.207622
4,6,0.05,100,-3.522501
5,6,0.1,100,-3.336743
6,6,0.01,150,-3.978599
7,6,0.05,150,-3.408314
8,6,0.1,150,-3.253472
9,8,0.01,50,-4.630581


In [52]:
final_model = CatBoostRegressor(random_state=42)
final_model.fit(X_train_scaled, y_train)

Learning rate set to 0.108945
0:	learn: 2.9096424	total: 205ms	remaining: 3m 25s
1:	learn: 2.7212783	total: 228ms	remaining: 1m 53s
2:	learn: 2.5593736	total: 255ms	remaining: 1m 24s
3:	learn: 2.4198871	total: 283ms	remaining: 1m 10s
4:	learn: 2.3016619	total: 312ms	remaining: 1m 2s
5:	learn: 2.2001601	total: 336ms	remaining: 55.7s
6:	learn: 2.1131309	total: 358ms	remaining: 50.9s
7:	learn: 2.0369686	total: 380ms	remaining: 47.1s
8:	learn: 1.9758931	total: 402ms	remaining: 44.3s
9:	learn: 1.9221934	total: 428ms	remaining: 42.4s
10:	learn: 1.8774764	total: 452ms	remaining: 40.7s
11:	learn: 1.8380442	total: 475ms	remaining: 39.1s
12:	learn: 1.8056917	total: 497ms	remaining: 37.8s
13:	learn: 1.7774853	total: 519ms	remaining: 36.6s
14:	learn: 1.7533586	total: 541ms	remaining: 35.5s
15:	learn: 1.7326109	total: 565ms	remaining: 34.7s
16:	learn: 1.7149831	total: 586ms	remaining: 33.9s
17:	learn: 1.6988828	total: 611ms	remaining: 33.3s
18:	learn: 1.6848510	total: 631ms	remaining: 32.6s
19:	lea

In [54]:
final_model1 = ExtraTreesRegressor(random_state=42)
final_model1.fit(X_train_scaled, y_train)

C:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ExtraTreesRegressor()

In [74]:
final_model2 = XGBRegressor(random_state=42)
final_model2.fit(X_train_scaled, y_train)

C:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
C:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\nasut\anaconda3\envs\Oppenheimer\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categori

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [59]:
final_model.score(X_test_scaled, y_test)

0.6381095617296877

In [56]:
final_model1.score(X_test_scaled, y_test)

0.6612935611685671

In [75]:
final_model2.score(X_test_scaled, y_test)

0.6496869701060428

In [60]:
import joblib

joblib.dump(final_model1, 'ETR.joblib')
joblib.dump(final_model, 'CR.joblib')

['CR.joblib']

In [61]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr

In [67]:
y_predicted = final_model1.predict(X_test_scaled)

In [68]:
mse = mean_squared_error(y_test, y_predicted)
mae = mean_absolute_error(y_test, y_predicted)
correlation = spearmanr(y_test, y_predicted)
print(f'MSE = {mse:.4f}, MAE = {mae:.4f}, Correlation = {correlation.correlation:.4f}, P-value of correlation = {correlation.pvalue:.4f}')

MSE = 3.0495, MAE = 1.3761, Correlation = 0.7614, P-value of correlation = 0.0000


In [69]:
X_test.index

Index(['2023-03-26 06:00:00', '2023-03-26 09:00:00', '2023-03-29 21:00:00',
       '2023-03-30 00:00:00', '2023-03-30 03:00:00', '2023-03-30 06:00:00',
       '2023-03-30 09:00:00', '2023-03-30 12:00:00', '2023-03-30 21:00:00',
       '2023-03-31 00:00:00',
       ...
       '2023-04-11 00:00:00', '2023-04-11 03:00:00', '2023-04-11 06:00:00',
       '2023-04-11 09:00:00', '2023-04-12 00:00:00', '2023-04-12 03:00:00',
       '2023-04-12 06:00:00', '2023-04-12 09:00:00', '2023-04-12 12:00:00',
       '2023-04-13 00:00:00'],
      dtype='object', name='Date', length=54428)

In [70]:
combined = pd.concat([y_test, pd.Series(y_predicted, index = y_test.index)], axis=1)
combined.columns = ['actual', 'prediction']
combined = combined.sort_index()

In [66]:
combined

,actual,prediction
Date,,
2021-02-27 15:00:00,28.7,28.536439
2021-02-27 15:00:00,28.7,28.723523
2021-02-27 15:00:00,24.2,23.480219
2021-02-27 15:00:00,25.1,25.471310
2021-02-27 15:00:00,28.5,29.580137
...,...,...
2023-04-13 21:00:00,25.3,26.768717
2023-04-13 21:00:00,26.4,27.979233
2023-04-13 21:00:00,25.3,23.906023


In [72]:
from sklearn.metrics import r2_score
r2_score(combined['actual'],combined['prediction'])

0.6612935611685671

In [81]:
# final_model1 = ExtraTreesRegressor(random_state=42, iterations=150, learning_rate=0.1)
# final_model1.fit(X_train_scaled, y_train)

TypeError: ExtraTreesRegressor.__init__() got an unexpected keyword argument 'iterations'

In [79]:
# Inisialisasi model ExtraTreesRegressor
model = ExtraTreesRegressor(random_state=42)

# Daftar hyperparameter yang akan diuji
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inisialisasi GridSearchCV dengan model dan daftar hyperparameter
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=5, verbose=3)

# Melakukan pencarian grid pada data
grid_search.fit(X_train_scaled, y_train)

# Menampilkan hyperparameter terbaik
best_params = grid_search.best_params_
print("Hyperparameter terbaik:", best_params)

# Menampilkan hasil evaluasi terbaik (misalnya MSE)
best_score = grid_search.best_score_
print("Skor terbaik (MSE):", -best_score)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


KeyboardInterrupt: 

In [ ]:
# save_model(best_model, 'weather_extra_trees_regressor')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['lokasi', 'con_prec.mm.',
                                              'mic_prec.mm.', 'sha_prec.mm.',
                                              'suhu2m.degC.', 'dew2m.degC.',
                                              'rh2m...', 'wspeed.m.s.',
                                              'wdir.deg.', 'lcloud...',
                                              'mcloud...', 'hcloud...',
                                              'surpre.Pa.', 'clmix.kg.kg.',
                                              'wamix.kg.kg.', 'outlr.W.m2.',
                                              'pblh.m.', 'lifcl.m.',
                                              'cape.j.kg.', 'mdbz', 't9...
                  TransformerWrapper(include=['Date'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('rest_encoding',